In [13]:
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
import logging
import utils as ut
#from utils import RNN, device

In [16]:
temp_test = ut.read_temps("data/tempAMAL_test.csv")
temp_train = ut.read_temps("data/tempAMAL_train.csv")
data.shape

TypeError: ufunc 'isnan' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''